<a href="https://colab.research.google.com/github/michtrofimov/transcriptomics.msu/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание

Загрузите прочтения всех экспериментов из статьи <a href="https://www.nature.com/articles/s41598-018-23226-4"><b>Evaluation of two main RNA-seq approaches for gene quantification in clinical RNA sequencing: polyA+ selection versus rRNA depletion</b></a>, в которой сравнивают два протокола подготовки библиотек для bulk RNA-Seq: при помощи деплеции рРНК (=<code>total</code>) или при помощи селекции polyA-транскриптов (=<code>polyA+</code>).

При выполнении задания используйте Nextflow pipeline.

In [8]:
!wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.0.0/sratoolkit.3.0.0-ubuntu64.tar.gz
!tar -xvzf sratoolkit.3.0.0-ubuntu64.tar.gz
!sratoolkit.3.0.0-ubuntu64/bin/vdb-config -i

--2022-10-13 15:38:53--  https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.0.0/sratoolkit.3.0.0-ubuntu64.tar.gz
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 165.112.9.230, 130.14.250.7, 2607:f220:41e:250::12, ...
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|165.112.9.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85777118 (82M) [application/x-gzip]
Saving to: ‘sratoolkit.3.0.0-ubuntu64.tar.gz.2’

sratoolkit.3.0.0-ub 100%[===================>]  81.80M  15.9MB/s    in 6.4s    

2022-10-13 15:39:01 (12.8 MB/s) - ‘sratoolkit.3.0.0-ubuntu64.tar.gz.2’ saved [85777118/85777118]

sratoolkit.3.0.0-ubuntu64/
sratoolkit.3.0.0-ubuntu64/bin/
sratoolkit.3.0.0-ubuntu64/bin/abi-dump
sratoolkit.3.0.0-ubuntu64/bin/abi-dump.3
sratoolkit.3.0.0-ubuntu64/bin/abi-load
sratoolkit.3.0.0-ubuntu64/bin/abi-load.3
sratoolkit.3.0.0-ubuntu64/bin/align-info
sratoolkit.3.0.0-ubuntu64/bin/align-info.3
sratoolkit.3.0.0-ubuntu64/bin/bam-load
srato

In [9]:
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
!unzip fastqc_v0.11.9.zip
!chmod +x FastQC/fastqc

--2022-10-13 15:39:24--  https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
Resolving www.bioinformatics.babraham.ac.uk (www.bioinformatics.babraham.ac.uk)... 149.155.133.4
Connecting to www.bioinformatics.babraham.ac.uk (www.bioinformatics.babraham.ac.uk)|149.155.133.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10249221 (9.8M) [application/zip]
Saving to: ‘fastqc_v0.11.9.zip’

fastqc_v0.11.9.zip  100%[===================>]   9.77M  3.53MB/s    in 2.8s    

2022-10-13 15:39:28 (3.53 MB/s) - ‘fastqc_v0.11.9.zip’ saved [10249221/10249221]

Archive:  fastqc_v0.11.9.zip
  inflating: FastQC/cisd-jhdf5.jar   
   creating: FastQC/Configuration/
  inflating: FastQC/Configuration/adapter_list.txt  
  inflating: FastQC/Configuration/contaminant_list.txt  
  inflating: FastQC/Configuration/limits.txt  
  inflating: FastQC/fastqc           
  inflating: FastQC/fastqc_icon.ico  
   creating: FastQC/Help/
   creating: FastQC/Help/1 Introductio

In [10]:
!pip3 install multiqc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 130 kB 67.5 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 237 kB 59.4 MB/s 
     |████████████████████████████████| 51 kB 5.7 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
  Created wheel for spectra: filename=spectra-0.0.11-py3-none-any.whl size=17488 sha256=82e65ee0357b057fd09efb3646f31f46f3d068a7292b9cbc5f88ea9886c14c8e
  Stored in directory: /root/.cache/pip/wheels/e3/c8/a4/5a566fafff827fcb5741589e42002686bb29990fbc7b262b5e
  Created wheel for colormath: filename=colormath-3.0.0-py3-none-any.whl size=39408 sha256=86004d3c956dea285457a7b48271d0feb0beb033bf6ac4f71a1211f722d33648
  Stored in directory: /root/.cache/pip/wheels/1a/d7/a7/458a4632ccc8bfe436f5e9db9fef1aa84e5d243a1b4b4c7b59
  Created wheel for lzstring: filenam

In [ ]:
!curl -fsSL https://get.nextflow.io | bash

In [ ]:
params.results_dir = "results/"
SRA_list = params.SRA.split(",")

log.info ""
log.info "  Q U A L I T Y   C O N T R O L  "
log.info "================================="
log.info "SRA number         : ${SRA_list}"
log.info "Results location   : ${params.results_dir}"

process DownloadFastQ {
  publishDir "${params.results_dir}"

  input:
    val sra

  output:
    path "${sra}/*"

  script:
    """
    /content/sratoolkit.3.0.0-ubuntu64/bin/fasterq-dump ${sra} -O ${sra}/
    """
}

process QC {
  input:
    path x

  output:
    path "qc/*"

  script:
    """
    mkdir qc
    /content/FastQC/fastqc -o qc $x
    """
}

process MultiQC {
  publishDir "${params.results_dir}"

  input:
    path x

  output:
    path "multiqc_report.html"

  script:
    """
    multiqc $x
    """
}

workflow {
  data = Channel.of( SRA_list )
  DownloadFastQ(data)
  QC( DownloadFastQ.out )
  MultiQC( QC.out.collect() )
}

In [12]:
!./nextflow run nextflow_pipeline.nf -with-report nextflow_report.html -with-dag flowchart.pdf --SRA SRR6410618,SRR6410617

N E X T F L O W  ~  version 22.10.0
Launching `nextflow_pipeline.nf` [cheesy_meninsky] DSL2 - revision: 30656d3ce8

  Q U A L I T Y   C O N T R O L  
SRA number         : [SRR6410618, SRR6410617]
Results location   : results/
[-        ] process > DownloadFastQ -
[-        ] process > QC            -
[-        ] process > MultiQC       -

[-        ] process > DownloadFastQ [  0%] 0 of 1
[-        ] process > QC            -
[-        ] process > MultiQC       -

executor >  local (2)
[8d/a7384c] process > DownloadFastQ (2) [  0%] 0 of 2
[-        ] process > QC                -
[-        ] process > MultiQC           -

executor >  local (2)
[8d/a7384c] process > DownloadFastQ (2) [ 50%] 1 of 2
[f4/1218f2] process > QC (1)            [  0%] 0 of 1
[-        ] process > MultiQC           -

executor >  local (3)
[8d/a7384c] process > DownloadFastQ (2) [ 50%] 1 of 2
[f4/1218f2] process > QC (1)            [  0%] 0 of 1
[-        ] process > MultiQC           -

executor >  local (4)
[e3